<a href="https://colab.research.google.com/github/juliandavidoviedo/Samsung-Innovation-Campus-Capstone/blob/main/Capstone_Project_JO_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar librerías necesarias
!pip install pandas requests scikit-learn nltk

# Importar librerías
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import os
import csv

# Descargar datos NLTK si es necesario
nltk.download('stopwords')
nltk.download('wordnet')

# Definir la URL de la API y la ruta de almacenamiento de datos
api_url = "https://www.datos.gov.co/resource/p6dx-8zbt.json"
data_path = "secop_data.csv"

# Función para actualizar y almacenar datos desde la API
def update_data():
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df.to_csv(data_path, index=False)
        print("Data updated successfully!")
    else:
        print(f"Error accessing API: {response.status_code}")

# Verificar si los datos existen, de lo contrario, actualizarlos
if not os.path.exists(data_path):
    update_data()

# Cargar datos preprocesados
df = pd.read_csv(data_path)

# Definir función de preprocesamiento de texto
def preprocess_text(text):
    """
    Preprocesa el texto de entrada para la búsqueda.
    Args:
        text (str): Texto a preprocesar.
    Returns:
        str: Texto preprocesado.
    """
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres no alfanuméricos
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios en blanco adicionales
    tokens = text.split()  # Tokenizar el texto
    lemmatizer = WordNetLemmatizer()  # Inicializar lematizador
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lematizar tokens
    stop_words = set(stopwords.words('spanish'))  # Obtener palabras vacías en español
    tokens = [token for token in tokens if token not in stop_words]  # Eliminar palabras vacías
    return ' '.join(tokens)  # Unir tokens preprocesados

# Preprocesar datos de descripciones y palabras clave
# Check if both columns exist before creating the 'processed_text' column
if 'nombre_del_procedimiento' in df.columns and 'descripci_n_del_procedimiento' in df.columns:
    # Create the 'processed_text' column by combining and handling missing values
    df['processed_text'] = df['nombre_del_procedimiento'].fillna('') + ' ' \
                          + df['descripci_n_del_procedimiento'].fillna('')

    # Apply any preprocessing or analysis to the 'processed_text' column as needed
    df['processed_text'] = df['processed_text'].apply(preprocess_text)
    print("LISTO")
else:
    # Handle the case where columns are missing
    print("Las columnas 'nombre_del_procedimiento' o 'descripci_n_del_procedimiento' no están disponibles. Se omitirá el preprocesamiento de texto.")

from sklearn.feature_extraction.text import TfidfVectorizer

# Define keywords
keywords = [
    "Innovación", "Transformación digital", "Internacionalización",
    "Logística", "Analítica de datos", "Machine learning",
    "Emprendimiento", "Habilidades blandas", "Planeación estratégica",
    "Estudio de mercado", "Inteligencia de negocio", "Marketing", "Comercio"
]
# Crear vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)  # Limitar el número de características
vectorizer.fit(df['processed_text'])

# Definir función para buscar convocatorias
def search_tenders(query):
    """
    Busca convocatorias relevantes para una consulta dada.
    Args:
        query (str): Consulta de búsqueda.
    Returns:
        pandas.DataFrame: DataFrame con resultados de búsqueda.
    """
    query_vec = vectorizer.transform([preprocess_text(query)])
    similarity_scores = cosine_similarity(query_vec, vectorizer.transform(df['processed_text'])).flatten()
    top_indices = similarity_scores.argsort()[-10:][::-1]
    results = df.iloc[top_indices].copy()
    results['puntuacion_similitud'] = similarity_scores[top_indices]
    return results

# Definir función para formatear la información de la convocatoria
def format_tender_info(results):
    """
    Formatea la información de las convocatorias encontradas.
    Args:
        results (pandas.DataFrame): DataFrame con resultados de búsqueda.
    Returns:
        list: Lista de resultados formateados.
    """
    formatted_results = []
    for _, row in results.iterrows():
        tender_info = {
            "Fecha de publicación": row.get('fecha_publicacion'),
            "Entidad": row.get('entidad'),
            "NIT Entidad": row.get('nit_entidad'),
            "Departamento Entidad": row.get('departamento_entidad'),
            "Ciudad Entidad": row.get('ciudad_entidad'),
            "Nombre del Procedimiento": row.get('nombre_del_procedimiento'),
            "Descripción del Procedimiento": row.get('descripci_n_del_procedimiento'),
            "Precio Base": row.get('precio_base'),
            "Modalidad de Contratación": row.get('modalidad_de_contratacion'),
            "Estado del Procedimiento": row.get('estado_del_procedimiento'),
            "Valor Total Adjudicación": row.get('valor_total_adjudicacion'),
            "Nombre del Adjudicador": row.get('nombre_del_adjudicador'),
            "Nombre del Proveedor": row.get('nombre_del_proveedor'),
            "NIT Proveedor Adjudicado": row.get('nit_del_proveedor_adjudicado'),
            "URL del Proceso": row.get('urlproceso')
        }
        formatted_result = "\n**Convocatoria:**\n"
        for key, value in tender_info.items():
            formatted_result += f"{key}: {value}\n"
        formatted_results.append(formatted_result)
    return formatted_results

# Definir función para generar CSV con los resultados formateados
def generate_csv(formatted_results, filename):
    """
    Genera un archivo CSV con los resultados formateados de la búsqueda.
    Args:
        formatted_results (list): Lista de resultados formateados.
        filename (str): Nombre del archivo CSV a generar.
    """
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        header = ["Fecha de publicación", "Entidad", "NIT Entidad", "Departamento Entidad",
                  "Ciudad Entidad", "Nombre del Procedimiento", "Descripción del Procedimiento",
                  "Precio Base", "Modalidad de Contratación", "Estado del Procedimiento",
                  "Valor Total Adjudicación", "Nombre del Adjudicador", "Nombre del Proveedor",
                  "NIT Proveedor Adjudicado", "URL del Proceso"]
        csv_writer.writerow(header)
        for result in formatted_results:
            tender_info = result.split('\n')[1:]  # Extract tender info from formatted result
            tender_data = [info.split(':')[1].strip() for info in tender_info if ':' in info]  # Extract values
            csv_writer.writerow(tender_data)
    print(f"CSV generado: {filename}")

# Simular la descarga de un archivo CSV (funcionalidad proyectada)
def download_csv(filename):
    """
    Simula la descarga de un archivo CSV mostrando un enlace de descarga (placeholder).
    Args:
        filename (str): Nombre del archivo CSV generado.
    """
    download_link = f"http://example.com/download/{filename}"  # Placeholder link
    print(f"Haga clic aquí para descargar el CSV: {download_link}")

# Función principal
def main():
    # Verificar si los datos existen, de lo contrario, actualizarlos
    if not os.path.exists(data_path):
        update_data()

    # Cargar datos preprocesados
    df = pd.read_csv(data_path)

    # Obtener palabras clave del usuario y realizar búsqueda
    user_keywords = input("Ingrese sus palabras clave de búsqueda: ")
    results = search_tenders(user_keywords)
    formatted_results = format_tender_info(results)

    # Imprimir resultados formateados
    print("Resultados de búsqueda:")
    if not formatted_results:
        print("No se encontraron convocatorias que coincidan con sus palabras clave.")
    else:
        for result in formatted_results:
            print(result)
            print("\n**¡Gracias por usar el modelo JO de Búsqueda Rápida!**")

    # Generar y descargar CSV
    csv_filename = "resultados_convocatorias.csv"
    generate_csv(formatted_results, csv_filename)
    download_csv(csv_filename)

# Ejecutar función principal
if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Data updated successfully!
LISTO
Ingrese sus palabras clave de búsqueda: Logística
Resultados de búsqueda:

**Convocatoria:**
Fecha de publicación: None
Entidad: SERVICIO NACIONAL DE APRENDIZAJE - SENA MAGDALENA
NIT Entidad: 8999990341
Departamento Entidad: Magdalena
Ciudad Entidad: Santa Marta
Nombre del Procedimiento: TITULADA
Descripción del Procedimiento: Prestar servicios personales de carácter temporal para impartir formación profesional programada por el Centro de Logística y Promoción Ecoturística Regional Magdalena en los Programas de Formación Titulada, Titulada Virtual, Complementaria y/o Complementaria Virtual
Precio Base: 589943520
Modalidad de Contratación: Contratación directa
Estado del Procedimiento: Seleccionado
Valor Total Adjudicación: 0
Nombre del Adjudicador: No Adjudicado
Nombre del Proveedor: IBIS
NIT Proveedor Adjudicado: 57444633
URL del Proceso: {'url': 'https://community.secop.gov.co/Public/Tendering/OpportunityDetail/Index?noticeUID=CO1.NTC.3872955'}


**¡G